<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# 9.02 Lab: Forecasting 

_Authors: Jeff Hale_

---

Let's practice time series forecasting.

---

### Data 

The Rossmann dataset.

---

Goal: forecast total daily sales for the Rossmann stores for 2015. This is a univariate time series problem. 

#### Get the data ready for modeling



#### Split the data into training and test sets. Make the 2015 data the test set.

#### Plot the training and test set sales over time.

## Baseline (naive) models

#### Make baseline models that are the mean, last value, and seasonal. Plot the predictions on the test set.

I strongly suggest you use sktime to make the forecasts and plot the predicitons.


#### Score your models on the test set using MAE.

## Holt-Winters

#### Make a model using Holt-Winters (triple exponential smoothing). Use statsmodels, pmdarima, or sktime. 

#### Plot the test set predictions vs. actuals.

#### Score your models on the test set using MAE. 

## SARIMA
#### Make a model using SARIMA. Use statsmodels, pmdarima, or sktime. 

#### Plot the test set predictions vs. actuals.

#### Score your models on the test set using MAE. 

#### Which model performs best?